In [ ]:
# One cell notebook: Lex section search + Azure OpenAI RAG
# -------------------------------------------------------

import os, json, requests
from dotenv import load_dotenv
from openai import AzureOpenAI

# ---------- 0. config --------------------------------------------------------
load_dotenv()

# Define the url of the Lex backend
LEX = "http://localhost:8000"
MODEL = "gpt-4o"

# Define the Azure OpenAI client
oai = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_KEY"),
    api_version = "2025-04-01-preview",
)

# ---------- 1. retriever -----------------------------------------------------
def search_sections(query: str, k: int = 3):
    """POST to send search parameters to the /legislation/section/search endpoint and return list of results"""
    r = requests.post(
        # one of REST API endpoints exposed by Lex for searching legislation sections
        url=f"{LEX}/legislation/section/search",
        # the request body which is the query to elastic search and the number of results to return
        json={"query": query, "size": k},
        # time after which the request is cancelled
        timeout=30,
    )
    # raise an exception if the request fails
    r.raise_for_status()
    return r.json()

# ---------- 2. RAG function --------------------------------------------------
def ask(question: str, k: int = 3) -> str:
    """
    Ask a question to the RAG system.
    """
    hits = search_sections(question, k)
    # if no hits are found, return a message
    if not hits:
        return "I couldn’t find any passages."

    # Build context string using the keys ('title', 'number', 'text', 'legislation_year')
    # ie format the results into context the LLM should use for the answer
    ctx = "\n\n".join(
        f"{h['title']} s.{h.get('number','?')} ({h['legislation_year']}): {h['text']}"
        for h in hits
    )
    # build the messages to send to the LLM, including system prompt, question and context
    messages = [
        {"role":"system",
         "content":"You are a UK public‑law assistant. Base answers strictly on the extracts provided."},
        {"role":"user",
         "content":f"Question: {question}\n\nExtracts:\n{ctx}"},
    ]

    # send the messages to the LLM
    resp = oai.chat.completions.create(model=MODEL, messages=messages)
    # return the answer
    return resp.choices[0].message.content

# ---------- 3. demo ----------------------------------------------------------
print( ask("What duties does the Climate Change Act impose on the Secretary of State?") )

/Users/radoslawkowalczyk/lex/.venv/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


The Climate Change Act imposes the following duties on the Secretary of State:

1. **Reports on Non-Devolved Provisions of the Act (s.97)**:
   - The Secretary of State must prepare and publish reports on the status of the provisions in Part 1 of the Act for each reporting period.
   - The report must include a statement that the Secretary of State is satisfied that the status of these provisions is appropriate.
   - The reports must address whether provisions are in force, and whether certain powers (e.g., bringing provisions into force, suspending or reviving provisions, altering expiry dates) have been exercised during the reporting period.
   - Reports are required every two months during the Act's substantive operational period.
   - Reports must be laid before Parliament. If a report is not prepared and published within 7 days after the reporting period ends, the Secretary of State must provide a written statement explaining the delay and publish it.

2. **Reports on Internationa